In [1]:
import torch
import os
import models
from charge_trainer import ChargeTrainer
from ocpmodels.common import logger
from ocpmodels.common.registry import registry
from ocpmodels.common.utils import setup_logging
from DeepDFT import densitymodel
from chg_utils import ProbeGraphAdder
setup_logging()

import matplotlib.pyplot as plt
from torch_geometric.data import Batch

In [2]:
!pip install wandb --upgrade
import wandb
wandb.login()
import pprint

2022-07-06 21:08:37 (ERROR): Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: emsunshine. Use `wandb login --relogin` to force relogin


In [3]:
sweep_config = {
    'method':'bayes',
    'metric':{'name':'val/charge_mae', 'goal':'minimize'},
    'parameters':{
        
        'num_interactions':{
            'distribution':'int_uniform',
            'max': 6,
            'min': 1,
        },
        
        'atom_channels':{
            'distribution':'q_log_uniform_values',
            'min': 16,
            'max': 128,
            'q': 8,
        },
        
        'probe_channels':{
            'distribution':'q_log_uniform_values',
            'min': 16,
            'max': 128,
            'q': 8,
        },
        
        'batch_size':{
            'distribution':'q_log_uniform_values',
            'min': 1,
            'max': 16,
            'q': 2,
        },
        
        'train_probes':{
            'distribution':'q_log_uniform_values',
            'min': 100,
            'max': 1000,
            'q': 100,
        },
        
        'atom_filters':{
            'distribution':'q_log_uniform_values',
            'min': 8,
            'max': 128,
            'q': 8,
        },
        
        'probe_filters':{
            'distribution':'q_log_uniform_values',
            'min': 8,
            'max': 128,
            'q': 8,
        },
        
        'atom_gaussians':{
            'distribution':'q_log_uniform_values',
            'min': 8,
            'max': 32,
            'q': 8,
        },
        
        'probe_gaussians':{
            'distribution':'q_log_uniform_values',
            'min': 8,
            'max': 128,
            'q': 8,
        },
        
        'cutoff': {
            'distribution':'int_uniform',
            'max': 6,
            'min': 3,
        },
    }
}

pprint.pprint(sweep_config)

{'method': 'bayes',
 'metric': {'goal': 'minimize', 'name': 'val/charge_mae'},
 'parameters': {'atom_channels': {'distribution': 'q_log_uniform_values',
                                  'max': 128,
                                  'min': 16,
                                  'q': 8},
                'atom_filters': {'distribution': 'q_log_uniform_values',
                                 'max': 128,
                                 'min': 8,
                                 'q': 8},
                'atom_gaussians': {'distribution': 'q_log_uniform_values',
                                   'max': 32,
                                   'min': 8,
                                   'q': 8},
                'batch_size': {'distribution': 'q_log_uniform_values',
                               'max': 16,
                               'min': 1,
                               'q': 2},
                'cutoff': {'distribution': 'int_uniform', 'max': 6, 'min': 3},
                'num_intera

In [4]:
sweep_id = wandb.sweep(sweep_config, project="charge-density-models-sweeps")
print(sweep_id)

Create sweep with ID: c4ve9o2z
Sweep URL: https://wandb.ai/emsunshine/charge-density-models-sweeps/sweeps/c4ve9o2z
c4ve9o2z


In [3]:
def train(config=None):
    with wandb.init(config=config):
        config = wandb.config
        
        task = {
            'dataset': 'lmdb',
            'description': 'Initial test of training on charges',
            'type': 'regression',
            'metric': ['charge_mse', 'charge_mae'],
            'primary_metric': 'charge_mae',
            'labels': ['charge_vals'],
        }
        
        model = {
            'name': 'charge_model',
            'num_interactions': config.num_interactions,
            'atom_channels': config.atom_channels,
            'probe_channels': config.probe_channels,

            'atom_model_config': {
                'name': 'schnet_charge',
                'num_filters':config.atom_filters,
                'num_gaussians':config.atom_gaussians,
                'cutoff':config.cutoff,
            },

            'probe_model_config': {
                'name': 'schnet_charge',
                'num_filters':config.probe_filters,
                'num_gaussians':config.probe_gaussians,
                'cutoff':config.cutoff,
            },
        }
        
        optimizer = {
            'optimizer': 'Adam',
            'batch_size': config.batch_size,
            'eval_batch_size': 10,
            'num_workers': 1,
            'lr_initial': 5e-4,
            'scheduler': "ReduceLROnPlateau",
            'mode': "min",
            'factor': 0.96,
            'patience': 1,
            'max_epochs': 300,
        }
        
        dataset = [
            {'src': '../chg/100/train', 'normalize_labels': False}, # train set 
            {'src': '../chg/100/val'}, # val set (optional)
            # {'src': train_src} # test set (optional - writes predictions to disk)
            ]
        
        trainer_config = {
            'trainer': 'charge',
            'identifier': 'sweep_run',
            'is_debug': False,
            'run_dir': './runs/',
            'print_every': 1,
            'seed': 2,
            'logger': 'wandb',
            'local_rank': 0,
            'amp': True,

            'cutoff': config.cutoff,
            'train_probes': config.train_probes,
            'val_probes': 1000,
            'test_probes': 1000,
        }
        
        trainer = registry.get_trainer_class(
            trainer_config['trainer'])(task = task,
                                       model = model,
                                       dataset = dataset,
                                       optimizer = optimizer,
                                       **trainer_config)
        
        trainer.train()

In [ ]:
wandb.agent('charge-density-models-sweeps/c4ve9o2z', train, count=100)

2022-07-06 21:08:40 (INFO): Starting sweep agent: entity=None, project=None, count=100


wandb: Agent Starting Run: 5pysvq5z with config:
wandb: 	atom_channels: 88
wandb: 	atom_filters: 16
wandb: 	atom_gaussians: 8
wandb: 	batch_size: 4
wandb: 	cutoff: 3
wandb: 	num_interactions: 4
wandb: 	probe_channels: 24
wandb: 	probe_filters: 24
wandb: 	probe_gaussians: 24
wandb: 	train_probes: 400
2022-07-06 21:08:40 (ERROR): Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: WARNING Ignored wandb.init() arg id when running a sweep.


amp: true
cmd:
  checkpoint_dir: ./runs/checkpoints/2022-07-06-21-09-20-sweep_run
  commit: 7d5240c
  identifier: sweep_run
  logs_dir: ./runs/logs/wandb/2022-07-06-21-09-20-sweep_run
  print_every: 1
  results_dir: ./runs/results/2022-07-06-21-09-20-sweep_run
  seed: 2
  timestamp_id: 2022-07-06-21-09-20-sweep_run
dataset:
  normalize_labels: false
  src: ../chg/100/train
gpus: 1
logger: wandb
model: charge_model
model_attributes:
  atom_channels: 88
  atom_model_config:
    cutoff: 3
    name: schnet_charge
    num_filters: 16
    num_gaussians: 8
  num_interactions: 4
  probe_channels: 24
  probe_model_config:
    cutoff: 3
    name: schnet_charge
    num_filters: 24
    num_gaussians: 24
noddp: false
optim:
  batch_size: 4
  eval_batch_size: 10
  factor: 0.96
  lr_initial: 0.0005
  max_epochs: 300
  mode: min
  num_workers: 1
  optimizer: Adam
  patience: 1
  scheduler: ReduceLROnPlateau
slurm: {}
task:
  dataset: lmdb
  description: Initial test of training on charges
  labels:
  

2022-07-06 21:08:54 (INFO): Loading dataset: lmdb
2022-07-06 21:08:54 (INFO): Loading model: charge_model
2022-07-06 21:08:54 (INFO): Loaded ChargeModel with 82579 parameters.
charge_mse: 1.94e-01, charge_mae: 3.73e-01, loss: 3.73e-01, lr: 5.00e-04, epoch: 4.35e-02, step: 1.00e+00
charge_mse: 1.29e-01, charge_mae: 3.34e-01, loss: 6.68e-01, lr: 5.00e-04, epoch: 8.70e-02, step: 2.00e+00
charge_mse: 2.05e-01, charge_mae: 3.43e-01, loss: 3.43e-01, lr: 5.00e-04, epoch: 1.30e-01, step: 3.00e+00
charge_mse: 2.86e-01, charge_mae: 3.37e-01, loss: 3.37e-01, lr: 5.00e-04, epoch: 1.74e-01, step: 4.00e+00
charge_mse: 1.05e-01, charge_mae: 2.72e-01, loss: 2.72e-01, lr: 5.00e-04, epoch: 2.17e-01, step: 5.00e+00
charge_mse: 1.14e-01, charge_mae: 2.71e-01, loss: 2.71e-01, lr: 5.00e-04, epoch: 2.61e-01, step: 6.00e+00
charge_mse: 3.94e-01, charge_mae: 2.58e-01, loss: 2.58e-01, lr: 5.00e-04, epoch: 3.04e-01, step: 7.00e+00
charge_mse: 1.90e-01, charge_mae: 2.47e-01, loss: 2.47e-01, lr: 5.00e-04, epoch: 3

...

2022-07-07 05:24:25 (INFO): charge_mse: 0.1010, charge_mae: 0.0285, loss: 0.0285, epoch: 76.0000


charge_mse: 4.67e-02, charge_mae: 2.65e-02, loss: 2.65e-02, lr: 2.12e-04, epoch: 7.61e+01, step: 1.14e+03
charge_mse: 2.56e-04, charge_mae: 9.21e-03, loss: 9.21e-03, lr: 2.12e-04, epoch: 7.61e+01, step: 1.14e+03
charge_mse: 9.57e-04, charge_mae: 9.58e-03, loss: 9.58e-03, lr: 2.12e-04, epoch: 7.62e+01, step: 1.14e+03
charge_mse: 5.14e-04, charge_mae: 8.05e-03, loss: 8.05e-03, lr: 2.12e-04, epoch: 7.63e+01, step: 1.14e+03
charge_mse: 3.01e-03, charge_mae: 9.08e-03, loss: 9.08e-03, lr: 2.12e-04, epoch: 7.63e+01, step: 1.14e+03
charge_mse: 7.34e-04, charge_mae: 7.88e-03, loss: 7.88e-03, lr: 2.12e-04, epoch: 7.64e+01, step: 1.15e+03
charge_mse: 5.66e-04, charge_mae: 1.02e-02, loss: 1.02e-02, lr: 2.12e-04, epoch: 7.65e+01, step: 1.15e+03
charge_mse: 2.46e-03, charge_mae: 1.40e-02, loss: 1.40e-02, lr: 2.12e-04, epoch: 7.65e+01, step: 1.15e+03
charge_mse: 5.06e-03, charge_mae: 1.35e-02, loss: 1.35e-02, lr: 2.12e-04, epoch: 7.66e+01, step: 1.15e+03
charge_mse: 4.97e-03, charge_mae: 1.04e-02, lo

device 0: 100%|██████████| 1/1 [00:07<00:00,  7.96s/it]

2022-07-07 05:25:40 (INFO): charge_mse: 0.0986, charge_mae: 0.0302, loss: 0.0302, epoch: 77.0000


charge_mse: 1.86e-03, charge_mae: 1.04e-02, loss: 1.04e-02, lr: 2.04e-04, epoch: 7.71e+01, step: 1.16e+03
charge_mse: 7.81e-04, charge_mae: 9.62e-03, loss: 9.62e-03, lr: 2.04e-04, epoch: 7.71e+01, step: 1.16e+03
